负责获取查询并返回相关文档

In [1]:
from langchain.document_loaders import TextLoader
# 定义文本加载器，指定读取的文件路径
loader = TextLoader("./documents/三国演义.txt")
documents=loader.load()
print(documents[0].metadata)


from langchain.text_splitter  import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=20,    
                                      separators=[
                                                "\n\n",
                                                "\n",
                                                " ",
                                                ".",
                                                ",",
                                                "\u200b",  # Zero-width space
                                                "\uff0c",  # Fullwidth comma
                                                "\u3001",  # Ideographic comma
                                                "\uff0e",  # Fullwidth full stop
                                                "\u3002",  # Ideographic full stop
                                                "",
                                            ],)
split_documents=text_splitter.split_documents(documents)

{'source': './documents/三国演义.txt'}


In [2]:
# 加载embedding模型
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

encode_kwargs = {"normalize_embeddings": False}
model_kwargs = {"device": "cuda:0"}
embeddings_model= HuggingFaceEmbeddings(
    model_name='/mnt/wushaogui/huggingface/shibing624/text2vec-base-chinese/',
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/tmp/ipykernel_14937/2788769724.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model= HuggingFaceEmbeddings(
/home/wushaogui/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wushaogui/anaconda3/envs/py39/lib/python3.9/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register

In [3]:
from langchain.vectorstores import Chroma

persist_directory="VectorStore" # 数据库保存路径
db = Chroma.from_documents(split_documents, embeddings_model, persist_directory=persist_directory)
db.persist()

/tmp/ipykernel_14937/2049267115.py:5: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [4]:
# 实例化一个检索器
retriever = db.as_retriever()
# 使用检索器查询查询文本
docs = retriever.invoke("吕布上马")
print(len(docs),docs[0])

4 page_content='，说吕布拱手来降，可乎？”卓大喜，观其人，乃虎贲中郎将李肃也。卓曰：“汝将何以说之？”肃曰：“某闻主公有名马一匹，号曰‘赤兔’，日行千里。须得此马，再用金珠，以利结其心；某更进说词：吕布必反丁原' metadata={'source': './documents/三国演义.txt'}


In [5]:
# 指定检索类型：最大边际相关性搜索
retriever = db.as_retriever(search_type="mmr")
# 使用检索器查询查询文本
docs = retriever.invoke("吕布上马")
print(len(docs),docs[0])

4 page_content='，说吕布拱手来降，可乎？”卓大喜，观其人，乃虎贲中郎将李肃也。卓曰：“汝将何以说之？”肃曰：“某闻主公有名马一匹，号曰‘赤兔’，日行千里。须得此马，再用金珠，以利结其心；某更进说词：吕布必反丁原' metadata={'source': './documents/三国演义.txt'}


In [15]:
# 指定相似度
retriever = db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 1e-6}
)
# 使用检索器查询查询文本
docs = retriever.invoke("吕布上马")
print(len(docs))

/home/wushaogui/anaconda3/envs/py39/lib/python3.9/site-packages/langchain_core/vectorstores/base.py:1083: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'source': './documents/三国演义.txt'}, page_content='，说吕布拱手来降，可乎？”卓大喜，观其人，乃虎贲中郎将李肃也。卓曰：“汝将何以说之？”肃曰：“某闻主公有名马一匹，号曰‘赤兔’，日行千里。须得此马，再用金珠，以利结其心；某更进说词：吕布必反丁原'), -191.54715317057455), (Document(metadata={'source': './documents/三国演义.txt'}, page_content='，说吕布拱手来降，可乎？”卓大喜，观其人，乃虎贲中郎将李肃也。卓曰：“汝将何以说之？”肃曰：“某闻主公有名马一匹，号曰‘赤兔’，日行千里。须得此马，再用金珠，以利结其心；某更进说词：吕布必反丁原'), -191.54715317057455), (Document(metadata={'source': './documents/三国演义.txt'}, page_content='次日，人报：“丁原引军城外搦战。”卓怒，引军同李儒出迎。两阵对圆，只见吕布顶束发金冠，披百花战袍，擐唐猊铠甲，系狮蛮宝带，纵马挺戟，随丁建阳出到阵前。建阳指卓骂曰：“国家不幸，阉官弄权'), -201.02395500324695), (Document(metadata={'source': './documents/三国演义.txt'}, page_content='次日，人报：“丁原引军城外搦战。”卓怒，引军同李儒出迎。两阵对圆，只见吕布顶束发金冠，披百花战袍，擐唐猊铠甲，系狮蛮宝带，纵马挺戟，随丁建阳出到阵前。建阳指卓骂曰：“国家不幸，阉官弄权'), -201.02395500324695)]
  self.vectorstore.similarity_search_with_relevance_scores(
No relevant docs

0


In [17]:
# 指定topk
retriever = db.as_retriever(search_kwargs={"k": 2})
# 使用检索器查询查询文本
docs = retriever.invoke("吕布上马")
print(len(docs),docs)

2 [Document(metadata={'source': './documents/三国演义.txt'}, page_content='，说吕布拱手来降，可乎？”卓大喜，观其人，乃虎贲中郎将李肃也。卓曰：“汝将何以说之？”肃曰：“某闻主公有名马一匹，号曰‘赤兔’，日行千里。须得此马，再用金珠，以利结其心；某更进说词：吕布必反丁原'), Document(metadata={'source': './documents/三国演义.txt'}, page_content='，说吕布拱手来降，可乎？”卓大喜，观其人，乃虎贲中郎将李肃也。卓曰：“汝将何以说之？”肃曰：“某闻主公有名马一匹，号曰‘赤兔’，日行千里。须得此马，再用金珠，以利结其心；某更进说词：吕布必反丁原')]


# 使用大模型生成上下文的MultiQueryRetriever

基于距离的向量数据库检索将查询嵌入高维空间，并根据距离度量找到类似的嵌入文档。但是，检索可能会因查询措辞的细微变化，或如果嵌入没有很好地捕捉数据的语义而产生不同的结果。提示工程/调整有时用于手动解决这些问题，但这可能很繁琐

MultiQueryRetriever 通过使用 LLM 为给定的用户输入查询生成多个不同视角的查询来自动化提示调整过程。对于每个查询，它都会检索一组相关文档，并获取所有查询的唯一并集以获得更大的潜在相关文档集。通过对同一问题的生成多种视角，MultiQueryRetriever 可以缓解基于距离检索的一些限制，并获得更丰富的结果

In [18]:
from langchain.retrievers.multi_query import MultiQueryRetriever

from langchain_ollama import OllamaLLM

# 初始化Ollama LLM，注意需要后台开启ollama服务
model_name = "llama3.1"
llm  = OllamaLLM(model=model_name)
print(f"OllamaLLM 初始化 {model_name} 完成")

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

OllamaLLM 初始化 llama3.1 完成


In [20]:
# 使用检索器查询查询文本
docs = retriever_from_llm.invoke("吕布骑的马叫什么？")
print(len(docs),docs)

5 [Document(metadata={'source': './documents/三国演义.txt'}, page_content='，以寄托作者的感慨，而书末以一首古风来概述本书内容。这首“卷首词”的词牌为“临江仙”，原为明代杨慎《二十一史弹词·说秦汉》中的开场词，清人毛宗岗修订《三国演义》时移作“开卷词”'), Document(metadata={'source': './documents/三国演义.txt'}, page_content='，因中国改朝换代的历史故事大多雷同，故此词作为《三国演义》的“卷首词”倒也恰当。'), Document(metadata={'source': './documents/三国演义.txt'}, page_content='，说吕布拱手来降，可乎？”卓大喜，观其人，乃虎贲中郎将李肃也。卓曰：“汝将何以说之？”肃曰：“某闻主公有名马一匹，号曰‘赤兔’，日行千里。须得此马，再用金珠，以利结其心；某更进说词：吕布必反丁原'), Document(metadata={'source': './documents/三国演义.txt'}, page_content='，不胜之喜。有良马一匹，日行千里，渡水登山，如履平地，名曰‘赤兔’，特献与贤弟，以助虎威。”布便令牵过来看，果然那马浑身上下，火炭般赤，无半根杂毛；从头至尾长一丈，从蹄至项高八尺；嘶喊咆哮'), Document(metadata={'source': './documents/三国演义.txt'}, page_content='三分好把姓名标——标：题写，记载。此句是说刘备、关羽、张飞将在魏、蜀、吴三国瓜分中国的争战中建立功勋，青史留名，永垂不朽。\u200b\n牒文——公文，文书。牒：古代可供书写的竹简。\u200b')]


# 使用大模型压缩检索到的数据

向量存储检索器是使用向量存储检索文档的检索器。它是一个围绕向量存储类的轻量级包装器，使其符合检索器接口。它使用向量存储实现的搜索方法（例如相似度搜索和 MMR）来查询向量存储中的文本


In [21]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_ollama import OllamaLLM

# 初始化Ollama LLM，注意需要后台开启ollama服务
model_name = "llama3.1"
llm  = OllamaLLM(model=model_name)
print(f"OllamaLLM 初始化 {model_name} 完成")

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

OllamaLLM 初始化 llama3.1 完成


In [22]:
# 使用检索器查询查询文本
docs = compression_retriever.invoke("吕布骑的马叫什么？")
print(len(docs),docs)

2 [Document(metadata={'source': './documents/三国演义.txt'}, page_content='乌牛白马——乌牛(黑牛)和白马都是祭祀或盟誓用的牺牲(供品)。《诗经·小雅·大田》：“来方禋祀，以其騂黑。”(毛传：“騂，牛也。黑，猪羊也。”)《史记·吕太后本纪》'), Document(metadata={'source': './documents/三国演义.txt'}, page_content='乌牛白马——乌牛(黑牛)和白马都是祭祀或盟誓用的牺牲(供品)。《诗经·小雅·大田》：“来方禋祀，以其騂黑。”(毛传：“騂，牛也。黑，猪羊也。”)《史记·吕太后本纪》：“高帝刑白马盟曰：‘非刘氏而王')]
